# Test biaslyze with disaster tweets data

Data source: https://www.kaggle.com/competitions/nlp-getting-started/overview

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
sys.path.append('/home/tobias/Repositories/biaslyze/')

In [163]:
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.metrics import accuracy_score
import matplotlib.pyplot as plt

## Load and prepare data

In [4]:
df = pd.read_csv("../data/disaster-tweets/train.csv"); df.head()

,id,keyword,location,text,target
0,1,NaN,NaN,Our Deeds are the Reason of this #earthquake M...,1
1,4,NaN,NaN,Forest fire near La Ronge Sask. Canada,1
2,5,NaN,NaN,All residents asked to 'shelter in place' are ...,1
3,6,NaN,NaN,"13,000 people receive #wildfires evacuation or...",1
4,7,NaN,NaN,Just got sent this photo from Ruby #Alaska as ...,1


In [151]:
df.shape

(7613, 5)

In [5]:
# replace urls
import re
url_regex = re.compile("(http|https)://[\w\-]+(\.[\w\-]+)+\S*")

df = df.replace(to_replace=url_regex, value='', regex=True)

## Train a model

In [34]:
clf = make_pipeline(TfidfVectorizer(min_df=10, max_features=10000, stop_words="english"), LogisticRegression(n_jobs=4))

In [35]:
clf.fit(df.text, df.target)

Pipeline(steps=[('tfidfvectorizer',
                 TfidfVectorizer(max_features=10000, min_df=10,
                                 stop_words='english')),
                ('logisticregression', LogisticRegression(n_jobs=4))])

In [36]:
train_pred = clf.predict(df.text)
print(accuracy_score(df.target, train_pred))

0.8418494680152371


## Test detection of concepts

In [37]:
from biaslyze.concept_detectors import KeywordConceptDetector
from biaslyze.evaluators import LimeBiasEvaluator

In [38]:
key_detect = KeywordConceptDetector()

In [39]:
detected_tweets = key_detect.detect(texts=df.text[:600])

2023-04-13 12:12:47.540 | INFO     | biaslyze.concept_detectors:detect:33 - Started keyword-based concept detection on 600 texts...
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 600/600 [00:00<00:00, 62455.51it/s]
2023-04-13 12:12:47.554 | INFO     | biaslyze.concept_detectors:detect:49 - Done. Found 412 texts with protected concepts.


In [40]:
len(detected_tweets)

412

In [41]:
detected_tweets

['Our Deeds are the Reason of this #earthquake May ALLAH Forgive us all',
 "All residents asked to 'shelter in place' are being notified by officers. No other evacuation or shelter in place orders are expected",
 '13,000 people receive #wildfires evacuation orders in California ',
 'Just got sent this photo from Ruby #Alaska as smoke from #wildfires pours into a school ',
 '#RockyFire Update => California Hwy. 20 closed in both directions due to Lake County fire - #CAfire #wildfires',
 '#flood #disaster Heavy rain causes flash flooding of streets in Manitou, Colorado Springs areas',
 "I'm on top of the hill and I can see a fire in the woods...",
 "There's an emergency evacuation happening now in the building across the street",
 "I'm afraid that the tornado is coming to our area...",
 'Three people died from the heat wave so far',
 '#Flood in Bago Myanmar #We arrived Bago',
 "What's up man?",
 'this is ridiculous....',
 'Love my girlfriend',
 'The end!',
 'We always try to bring the he

## Test LIME based bias detection with keywords

In [127]:
from biaslyze.bias_detectors import LimeKeywordBiasDetector

In [458]:
bias_detector = LimeKeywordBiasDetector(bias_evaluator=LimeBiasEvaluator(n_lime_samples=500), n_top_keywords=30, use_tokenizer=True)

In [459]:
detection_res = bias_detector.detect(texts=df.text.sample(frac=0.3), predict_func=clf.predict_proba)

2023-04-14 11:58:11.172 | INFO     | biaslyze.concept_detectors:detect:33 - Started keyword-based concept detection on 2284 texts...
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2284/2284 [00:22<00:00, 99.97it/s]
2023-04-14 11:58:34.024 | INFO     | biaslyze.concept_detectors:detect:49 - Done. Found 368 texts with protected concepts.
2023-04-14 11:58:34.025 | INFO     | biaslyze.evaluators:evaluate:44 - Started bias detection on 368 samples...
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 368/368 [00:59<00:00,  6.21it/s]


In [460]:
detection_res.summary()

Detected 340 samples with potential issues.
    Potentially problematic concepts detected: [('gender', 211), ('nationality', 127), ('religion', 12)]
    Based on keywords: [('man', 38), ('california', 37), ('he', 36), ('his', 29), ('her', 25), ('she', 16), ('japan', 13), ('israeli', 11), ('girl', 11), ('mom', 11), ('saudi', 10), ('boy', 10), ('china', 9), ('woman', 8), ('men', 8), ('guy', 8), ('pakistan', 8), ('christian', 7), ('russian', 7), ('women', 7)].


In [461]:
detection_res.details(group_by_concept=True)

Concept: gender
[{'reason': ['woman'],
  'text': 'Mass murderer Che Guevara greeting a woman in North Korea '},
 {'reason': ['father', 'son'],
  'text': '@RaidersReporter @957thegame together we can destroy the Emperor '
          'and rule the galaxy Father and Son!'},
 {'reason': ['wife', 'her', 'his'],
  'text': "Bigamist and his 'first' wife are charged in the deaths of his "
          "'second' pregnant wife her child 8 her mothe... "},
 {'reason': ['his'],
  'text': 'Purple Heart Vet Finds Jihad Threat on His Car at Mall: \x89ÛÏAll '
          'of you Islamaphobe vets... '},
 {'reason': ['man'],
  'text': 'Horrible Accident |  Man Died In Wings of AirplaneåÊ(29-07-2015) '},
 {'reason': ['man'],
  'text': 'Parents are taking their kids to Burning Man and one 11 year old '
          "thinks it's 'better than... "},
 {'reason': ['woman'],
  'text': 'Pandemonium In Aba As Woman Delivers Baby Without Face '
          '(Photos)....  '},
 {'reason': ['man'],
  'text': 'to whomever is hi

In [473]:
from bokeh.io import show, output_notebook

output_notebook()

dashboard = detection_res.dashboard()
show(dashboard)

Loading BokehJS ...

In [465]:
pos_dashboard = detection_res.dashboard(use_position=True)
show(pos_dashboard)

## Testing a sentiment analysis model from huggingface

In [25]:
from transformers import pipeline
from torch.utils.data import Dataset


classifier = pipeline(
    model="distilbert-base-uncased-finetuned-sst-2-english",
    top_k=None,
    padding=True,
    truncation=True
)

In [26]:
class MyDataset(Dataset):
    def __init__(self, data):
        super().__init__()
        self.data = data

    def __len__(self):
        return len(self.data)

    def __getitem__(self, i):
        return self.data[i]


def predict_sentiment(texts):
    data = MyDataset(texts)
    proba = []
    for res in classifier(data):
        proba_array = []
        for p in sorted(res, key=lambda d: d['label'], reverse=True):
            proba_array.append(p.get("score"))
        proba.append(np.array(proba_array))
    return np.array(proba) / np.array(proba).sum(axis=1)[:,None]

In [27]:
bias_detector = LimeKeywordBiasDetector(
    bias_evaluator=LimeBiasEvaluator(n_lime_samples=500),
    n_top_keywords=10,
    use_tokenizer=True
)

In [28]:
test_texts = detected_tweets[:10]
detection_res = bias_detector.detect(texts=test_texts, predict_func=predict_sentiment)

2023-04-13 10:09:41.105 | INFO     | biaslyze.concept_detectors:detect:33 - Started keyword-based concept detection on 10 texts...
100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 10/10 [00:00<00:00, 109.41it/s]
2023-04-13 10:09:41.200 | INFO     | biaslyze.concept_detectors:detect:49 - Done. Found 2 texts with protected concepts.
2023-04-13 10:09:41.201 | INFO     | biaslyze.evaluators:evaluate:42 - Started bias detection on 2 samples...
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:38<00:00, 19.30s/it]


In [29]:
detection_res.summary()

Detected 1 samples with potential issues.
    Potentially problematic concepts detected: [('nationality', 1)]
    Based on keywords: [('california', 1)].


In [30]:
detection_res.details(group_by_concept=True)

Concept: nationality
[{'reason': ['california'],
  'text': '13,000 people receive #wildfires evacuation orders in California '}]


## !! Very Experimental !!: Test masked language model based bias detection with keywords

In [31]:
from biaslyze.bias_detectors import MaskedKeywordBiasDetector

In [466]:
bias_detector = MaskedKeywordBiasDetector(n_resample_keywords=20, use_tokenizer=True)

In [467]:
masked_detection_res = bias_detector.detect(texts=df.sample(1000).text, predict_func=clf.predict_proba)

2023-04-14 11:59:35.431 | INFO     | biaslyze.concept_detectors:detect:33 - Started keyword-based concept detection on 1000 texts...
100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1000/1000 [00:10<00:00, 96.34it/s]
2023-04-14 11:59:45.817 | INFO     | biaslyze.concept_detectors:detect:49 - Done. Found 170 texts with protected concepts.
2023-04-14 11:59:45.817 | INFO     | biaslyze.evaluators:evaluate:109 - Started bias detection on 170 samples...
100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 170/170 [00:10<00:00, 17.00it/s]


In [468]:
masked_detection_res.summary()

Detected 91 samples with potential issues.
    Potentially problematic concepts detected: [('gender', 49), ('nationality', 42), ('religion', 1)]
    Based on keywords: [('he', 12), ('man', 10), ('california', 10), ('his', 8), ('japan', 7), ('she', 6), ('her', 5), ('boy', 4), ('women', 3), ('turkish', 3), ('china', 3), ('woman', 3), ('german', 3), ('guy', 2), ('ladies', 2), ('daughter', 2), ('myanmar', 2), ('men', 2), ('russia', 2), ('palestine', 2)].


In [469]:
masked_detection_res.details()

''Another white guy trying to mass murder people for no apparent reason just because let me guess he's mentally ill blah blah blah #Antioch'' might contain bias ['gender']; reasons: ['he', 'guy']
''2015 new fashion ladies gold watch waterproof WeiQin famous brand michel quartz de lujo caÛ_  '' might contain bias ['gender']; reasons: ['ladies']
''@O_Magazine satan's daughter shadow warrior in 50ft women aka transgender mode ps nyc is about to fold extra extra center of bioterrorism'' might contain bias ['gender']; reasons: ['women', 'daughter']
''Turkish troops killed in Kurdish militant 'suicide attack' '' might contain bias ['nationality']; reasons: ['turkish']
''Chinese rescue team arrives in Myanmar to help flood victims  Sittway '' might contain bias ['nationality']; reasons: ['myanmar', 'chinese']
''@APANO55 @JamesMelville 99% of Scientists donÛªt believe in Catastrophic Man-Made Global Warming only the deluded do.'' might contain bias ['gender']; reasons: ['man', 'man']
''ÛÏ@B

In [471]:
from bokeh.io import show, output_notebook

output_notebook()

dashboard = masked_detection_res.dashboard()
show(dashboard)

Loading BokehJS ...